### Applied Data Science Capstone Week-3 Assignment.
### Segmenting and Clustering Neighborhoods in Toronto

##  Question - 1
### Build a dataframe of the postal code of each neighborhood along with borough names and neighborhood names.

#### 1, Import Libraries

In [1]:
# install folium
!pip install folium

import numpy as np                        # library for vectorized computation
import pandas as pd                       # library to process data as dataframes

import requests                           # library to handle requests
from bs4 import BeautifulSoup             # library to parse HTML and XML documents

import json                               # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim     # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import folium
import folium                              # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 94 kB 7.4 MB/s  eta 0:00:01
Libraries imported.


#### 2, Scrape data from Wikipedia page to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe.

In [2]:
# get request
data_t = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data_t, 'html.parser')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [3]:
# append the data into the respective lists, avoid new lines \n in cell
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [4]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### 3, Drop cells with a borough that is Not assigned. Only process the cells that have an assigned borough. 

In [5]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head(6)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


#### 4, More than one neighborhood can exist in the same Borough. Combine (group) neighborhoods in the same Borough separated by comma.

In [6]:
# group/combine neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head(6)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village


#### 5, If a cell has a borough but a Not assigned  neighborhood, then assign borough to neighborhood.

In [7]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Final step - use .shape method to print the number of rows of your cleaned dataframe.

In [8]:
# Final step, use .shape method to print the number of rows of your cleaned dataframe.
toronto_df_grouped.shape

(103, 3)

## Question - 2
### get the geographical coordinates of the neighborhoods using the Geocoder package 
### or csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

#### 1, read file Geospatial_data into dataframe

In [9]:
# load the coordinates from the csv file on "http://cocl.us/Geospatial_data"
coordinates_df = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates_df.head(6)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476


#### 2, Rename 'Postal Code' to PostalCode

In [11]:
# rename the column "PostalCode"
coordinates_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates_df.head(6)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476


#### 3, Merge two tables to get the coordinates (coordinates_df & toronto_df_grouped)

In [12]:
#merge two tables (coordinates_df & toronto_df_grouped) on the column "PostalCode" and dispaly 11 rows to verify
toronto_df_new = toronto_df_grouped.merge(coordinates_df, on="PostalCode", how="left")
toronto_df_new.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Final - use .shape method to print the number of rows and columns

In [13]:
## use .shape method to print the number of rows and columns
toronto_df_new.shape

(103, 5)

##   Question - 3
####  Explore and cluster the neighborhoods in Toronto. 
####  Work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

#### 1, step1 - use geopy library to get the latitude and longitude values of Toronto

In [14]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator  = Nominatim(user_agent="nm-application") # specify appliaction name
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 2, Create a map of Toronto with neighborhoods superimposed on top

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### 3, Filter only boroughs that contain the word Toronto

In [16]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

#### 4, Create new dataframe with Boroughs that contain the word Toronto

In [17]:
#create new df with Toronto
toronto_df_new2 = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)]
toronto_df_new2.head(6) 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197


#### 5, Foursquare app: Use the Foursquare api to explore the neighborhoods

In [18]:
CLIENT_ID = 'JXSHDM3INM2TUNT1Z3MVT2C3JUJC3LWVNOA1UTQ5A04BZEXZ' # your Foursquare ID
CLIENT_SECRET = 'T2YAIOY2B0YULJ1YJZXXTE4CCCAFZEVJ23XUQICT2AB4ZUA3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # a default foursquare api limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JXSHDM3INM2TUNT1Z3MVT2C3JUJC3LWVNOA1UTQ5A04BZEXZ
CLIENT_SECRET:T2YAIOY2B0YULJ1YJZXXTE4CCCAFZEVJ23XUQICT2AB4ZUA3


#### 6, Get the top 100 venues that are within a radius of 500 meters radious.

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###  7.  Final - execute function above getNearbyVenues and get list of venues in Toronto.

In [20]:
# type your answer here
toronto_venues=None
toronto_venues = getNearbyVenues(names=toronto_df_new2['Neighborhood'],
                                  latitudes=toronto_df_new2['Latitude'],
                                   longitudes=toronto_df_new2['Longitude']
                                 )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

### 8, Display head of dataframe toronto_venues (Final)

In [21]:
# dataframe toronto_venues
toronto_venues.head(6)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
